# EAD impact of a variation margin threshold

We set up a single trade portfolio in a vm margined collateral agreement with a VM threshold:

In [1]:
import QuantLib as ql
from collateralAgreement.collateralAgreement import CollateralAgreement, Margining, InitialMargining
from jupyterUtils import export
from marketdata import init_marketdata
from instruments.interestRateInstrument.irs import IRS
from marketdata.interestRateIndices import InterestRateIndex
from sa_ccr.sa_ccr import SA_CCR
asdf = 1

In [2]:
irs = IRS(notional=2000000000,
          index=InterestRateIndex.USDLIBOR3M,
          timeToSwapStart=ql.Period(2, ql.Days),
          timeToSwapEnd=ql.Period(10, ql.Years),
          fixed_rate=0.024)

irs.get_price()

1643257.9608547091

In [3]:
ca_10mn = CollateralAgreement(margining=Margining.MARGINED,
                              initialMargining=InitialMargining.NO_IM,
                              threshold_vm=10000000)
ca_10mn.link_sa_ccr_instance(SA_CCR(ca_10mn))
ca_10mn.add_trades(irs)

ca_1mn = CollateralAgreement(margining=Margining.MARGINED,
                             initialMargining=InitialMargining.NO_IM,
                             threshold_vm=1000000)
ca_1mn.link_sa_ccr_instance(SA_CCR(ca_1mn))
ca_1mn.add_trades(irs)

ca_0 = CollateralAgreement(margining=Margining.MARGINED,
                           initialMargining=InitialMargining.NO_IM,
                           threshold_vm=0)
ca_0.link_sa_ccr_instance(SA_CCR(ca_0))
ca_0.add_trades(irs)


When calculating the EAD, PFE and RC of the three portfolios we can see that a VM threshold only impacts the RC and that inclusion of a variation margin threshold simply increases the EAD by $1.4*\text{VM threshold}$.

In [4]:
print(ca_10mn.get_sa_ccr_model().get_ead())
print(ca_1mn.get_sa_ccr_model().get_ead())
print(ca_0.get_sa_ccr_model().get_ead())

47028094.49124615
34428094.49124615
33028094.491246153


In [5]:
print(ca_10mn.get_sa_ccr_model().get_rc())
print(ca_1mn.get_sa_ccr_model().get_rc())
print(ca_0.get_sa_ccr_model().get_rc())

10000000.0
1000000.0
0.0


In [6]:
print(ca_10mn.get_sa_ccr_model().get_pfe())
print(ca_1mn.get_sa_ccr_model().get_pfe())
print(ca_0.get_sa_ccr_model().get_pfe())

23591496.065175824
23591496.065175824
23591496.065175824


This impact of the EAD even persists if the portfolio is empty.

In [7]:
ca_10mn.remove_all_trades()
ca_10mn.get_sa_ccr_model().get_ead()

14000000.0

In [9]:
export('Exemplary SA-CCR allocation considering a variation margin threshold.ipynb')